In [1]:
import requests
import re
import pandas as pd

### Query Function:
>This function will send our text to bern2 using their API to get the text labeled

In [2]:
text = "Autophagy maintains tumour growth through circulating the great arginine."
text1 = "X-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. She was initially treated with ice, an ace wrap, crutches and mild pain medications (Tylenol with codeine)"
text2 = "It is a skin disease causing much itchiness. Scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling."
texts = [text,text1,text2]

In [3]:
coms = pd.DataFrame({'comments':texts})
coms.reset_index(inplace=True)
coms.rename(columns = {"index":"idx"},inplace=True)
coms.idx = coms.idx.astype(str)
coms['comidx'] = coms.idx +': '+coms.comments+'. ::'
coms.comidx = coms.comidx.str.lower().str.replace(r'\(|\)',',',regex=True)
coms

,idx,comments,comidx
0,0,Autophagy maintains tumour growth through circ...,0: autophagy maintains tumour growth through c...
1,1,X-rays were negative and physical assessment d...,1: x-rays were negative and physical assessmen...
2,2,It is a skin disease causing much itchiness. S...,2: it is a skin disease causing much itchiness...


In [4]:
texty = ' '.join(coms.comidx.tolist())

In [5]:
def query_plain(text, url="http://bern2.korea.ac.kr/plain"):
    return requests.post(url, json={'text': text}).json()

# if __name__ == '__main__':
#     text = "Autophagy maintains tumour growth through circulating arginine"
#     print(query_plain(text))

#### Get Start and end index of each entry

In [6]:
texty

'0: autophagy maintains tumour growth through circulating the great arginine.. :: 1: x-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. she was initially treated with ice, an ace wrap, crutches and mild pain medications ,tylenol with codeine,. :: 2: it is a skin disease causing much itchiness. scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling.. ::'

In [7]:
string = texty
pattern = r'(\d:.*?)::'
dfi = pd.DataFrame()
for o in re.findall(pattern,string):
        print(o)
        print()
        match=(re.search(o, string))
        print(match)
                #Getting the start and end index in tuple format using match.span()
        text = o
        start = match.start()
        end = match.end()
        span = match.span()
        dfi = dfi.append({'text':text,'start':start,'end':end,'span':span},ignore_index=True)
        print ("start and end index", match.span())
dfi   

0: autophagy maintains tumour growth through circulating the great arginine.. 

<re.Match object; span=(0, 78), match='0: autophagy maintains tumour growth through circ>
start and end index (0, 78)
1: x-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. she was initially treated with ice, an ace wrap, crutches and mild pain medications ,tylenol with codeine,. 

<re.Match object; span=(81, 303), match='1: x-rays were negative and physical assessment d>
start and end index (81, 303)
2: it is a skin disease causing much itchiness. scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling.. 

<re.Match object; span=(306, 448), match='2: it is a skin disease causing much itchiness. s>
start and end index (306, 448)


,text,start,end,span
0,0: autophagy maintains tumour growth through c...,0.0,78.0,"(0, 78)"
1,1: x-rays were negative and physical assessmen...,81.0,303.0,"(81, 303)"
2,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"


In [8]:
string = texty
pattern = r'(\d:.*?)::'
dfi = pd.DataFrame()
for o in re.findall(pattern,string):
        print(o)
        print()
        match=(re.search(o, string))
        print(match)
                #Getting the start and end index in tuple format using match.span()
        text = o
        start = match.start()
        end = match.end()
        span = match.span()
        dfi = dfi.append({'text':text,'start':start,'end':end,'span':span},ignore_index=True)
        print ("start and end index", match.span())
dfi   

0: autophagy maintains tumour growth through circulating the great arginine.. 

<re.Match object; span=(0, 78), match='0: autophagy maintains tumour growth through circ>
start and end index (0, 78)
1: x-rays were negative and physical assessment determined soft tissue damage to the lateral aspect of her ankle. she was initially treated with ice, an ace wrap, crutches and mild pain medications ,tylenol with codeine,. 

<re.Match object; span=(81, 303), match='1: x-rays were negative and physical assessment d>
start and end index (81, 303)
2: it is a skin disease causing much itchiness. scratching leads to redness, swelling, cracking, weeping clear fluid, crusting, and scaling.. 

<re.Match object; span=(306, 448), match='2: it is a skin disease causing much itchiness. s>
start and end index (306, 448)


,text,start,end,span
0,0: autophagy maintains tumour growth through c...,0.0,78.0,"(0, 78)"
1,1: x-rays were negative and physical assessmen...,81.0,303.0,"(81, 303)"
2,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"


# Send combined text to bern2

In [11]:
output = query_plain(texty)

In [12]:
dfa = pd.DataFrame(output['annotations'])

### Combine original text chunk with returned labeled text from BERN
>We sent all separate text documents as one big text document to bern.  Now we'll re-separate the labeled text to show which biomedical words were in which documents.

In [13]:
# create str_end col as type str
dfa.span = dfa.span.astype(str)
dfa ['str_end'] = dfa.span.str.replace(r".*'end': (\d+)}",r"\1",regex=True)

dfa.str_end = dfa.str_end.astype(int)  # make str end as type str

#add dfi_idx col
dfi.reset_index(inplace=True)
dfi.rename(columns={'index':'dfi_idx'},inplace=True)

for o,m in zip(dfi.index,dfi.span):  #add dfi_idx col to dfa 
    x,y = m #open span tuple
    #write conditions for specific df rows
    conds = (dfa.str_end > x) & (dfa.str_end < y)
    dfa.loc[conds,'dfi_idx'] = o #save the index of the dfi span that fits to dfa

# dfa.merge(dfi, left_on='dfi_idx',right_index=True)
df = dfa.merge(dfi, left_on='dfi_idx',right_on='dfi_idx')
df

,id,is_neural_normalized,mention,obj,prob,span_x,str_end,dfi_idx,text,start,end,span_y
0,[mesh:D009369],False,tumour,disease,0.999996,"{'begin': 23, 'end': 29}",29,0.0,0: autophagy maintains tumour growth through c...,0.0,78.0,"(0, 78)"
1,[mesh:D001120],False,arginine,drug,0.994983,"{'begin': 67, 'end': 75}",75,0.0,0: autophagy maintains tumour growth through c...,0.0,78.0,"(0, 78)"
2,[mesh:D000082],False,tylenol,drug,0.997189,"{'begin': 280, 'end': 287}",287,1.0,1: x-rays were negative and physical assessmen...,81.0,303.0,"(81, 303)"
3,[mesh:D003061],False,codeine,drug,0.946703,"{'begin': 293, 'end': 300}",300,1.0,1: x-rays were negative and physical assessmen...,81.0,303.0,"(81, 303)"
4,[mesh:D012871],False,skin disease,disease,0.999603,"{'begin': 317, 'end': 329}",329,2.0,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"
5,[mesh:D011537],False,itchiness,disease,0.986398,"{'begin': 343, 'end': 352}",352,2.0,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"
6,[mesh:D000080822],False,redness,disease,0.964273,"{'begin': 374, 'end': 381}",381,2.0,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"
7,[mesh:D004487],True,swelling,disease,0.988147,"{'begin': 383, 'end': 391}",391,2.0,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"
8,[mesh:D012135],True,cracking,disease,0.974428,"{'begin': 393, 'end': 401}",401,2.0,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"
9,[mesh:D002862],True,crusting,disease,0.995957,"{'begin': 424, 'end': 432}",432,2.0,2: it is a skin disease causing much itchiness...,306.0,448.0,"(306, 448)"
